<a href="https://colab.research.google.com/github/alexlauforgithub/mystuffs/blob/master/%E8%B2%A1%E9%87%91%E8%B3%87%E8%A8%8A%E7%88%AC%E8%9F%B2%E6%A0%B8%E5%BF%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import os
import time
import datetime

## 爬取大盤歷史資料

In [ ]:
# 日期設定
first_date = datetime.datetime.today().replace(day=1).strftime('%Y-%m-%d')
start_date = '2000-01-01'
date_list = pd.date_range(start_date, first_date, freq='MS')
date_list = date_list.strftime('%Y%m%d').tolist()

In [7]:
i='20210805'

url = 'https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=csv&date='+ '20210701' 
res = requests.get(url)
text = res.text

if len(text) == 0:
    None

else:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                            for ele in da.split('","')])
    
    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[1])[2:].dropna().reset_index(drop=True)
    df.columns = ['date', 'open', 'high', 'low', 'close']
    #民國年轉西元年
    for ix in range(len(df['date'])):
        df['date'].iloc[ix]=df['date'].iloc[ix].replace(df['date'].iloc[ix][0:3], str(int(df['date'].iloc[ix][0:3]) + 1911))
    # 轉成數值
    df['date'] = df['date'].str.replace('/', '').astype(int)

## 爬取股價歷史資料

In [ ]:
# 日期設定
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = '2004-02-11' # 證交所最早提供的日期
date_list = pd.date_range(start=start_date ,end=today_date).strftime('%Y%m%d').tolist()

In [45]:
i='20210805'
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ i + '&type=ALLBUT0999'
res = requests.get(url)
text = res.text

if len(text) == 0:
    print(i +" is holiday!")
else:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])
    
    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df['日期'] = i
    df = df[['日期', '證券代號', '證券名稱', '開盤價', '最高價', '最低價', '收盤價', '成交股數', '成交金額']]
    df.columns = ['date_num', 'code', 'name', 'open', 'high', 'low', 'close', 'volume_share', 'volume_value']
    
    # 取股票代碼4位數字
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)

    df = df.replace('--', np.nan)
    df = df.replace(',','', regex=True)
    num_feat = ['date_num', 'open', 'high', 'low', 'close','volume_share', 'volume_value']
    df[num_feat] = df[num_feat].astype(float)

## 爬取證交所籌碼資料

In [ ]:
# 日期設定
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = '2014-12-01' # 證交所最早提供的日期
date_list = pd.date_range(start=start_date ,end=today_date).strftime('%Y%m%d').tolist()

In [ ]:
## 施工中

In [54]:
# 爬取證交所籌碼資料
i = '20130701'

url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ i + '&selectType=ALLBUT0999'
res = requests.get(url)
text = res.text
ix = int(i)

if len(text) == 0 or text == '\r\n':
    print('Ho!')




elif ix < 20141201:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 12 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df.columns = ['code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_BUY', 'DL_SELL', 'TOP3_TOTAL']
    df['date'] = i
    df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_BUY', 'DL_SELL', 'TOP3_TOTAL']]
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)


elif ix < 20171218 and ix >= 20141201 :
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df.columns = ['code', 'name', 'fi_buy', 'fi_sell', 'fi_diff', 'sitc_buy', 'sitc_sell', 'sitc_di', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']
    df['date'] = i
    df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']]
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)

elif ix >= 20171218:
    # 把爬下來的資料整理乾淨
    cleaned_data = []
    for da in text.split('\n'):
        if len(da.split('","')) == 19 and da.split('","')[0][0] != '=':
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                  for ele in da.split('","')])

    # 輸出成表格並呈現在excel上
    df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
    df.columns = ['code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'FI_SELF_BUY', 'FI_SELF_SELL', 'FI_SELF_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']
    df['date'] = i
    df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'FI_SELF_BUY', 'FI_SELF_SELL', 'FI_SELF_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']]
    df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)

In [55]:
df

,date,code,name,FI_BUY,FI_SELL,FI_TOTAL,SIT_BUY,SIT_SELL,SIT_TOTAL,DL_TOTAL,DL_BUY,DL_SELL,TOP3_TOTAL
0,20130701,2303,聯電,"26,269,000","28,204,370","-1,935,370","10,552,000","19,000","10,533,000","2,747,000","6,839,000","4,092,000","11,344,630"
1,20130701,3474,華亞科,"13,283,000","4,303,000","8,980,000",0,0,0,0,0,0,"8,980,000"
2,20130701,2883,開發金,"9,702,000","6,382,800","3,319,200",0,"21,000","-21,000","4,967,000","7,624,000","2,657,000","8,265,200"
3,20130701,2325,矽品,"11,744,000","6,363,369","5,380,631",0,"5,000","-5,000","2,740,000","3,223,000","483,000","8,115,631"
4,20130701,2887,台新金,"8,206,000","2,547,000","5,659,000","1,700,000","9,000","1,691,000","212,000","546,000","334,000","7,562,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,20130701,6116,彩晶,"94,000","7,815,394","-7,721,394",0,0,0,"195,000","200,000","5,000","-7,526,394"
741,20130701,2498,宏達電,"2,457,087","10,771,270","-8,314,183",0,"95,000","-95,000","-127,015","261,000","388,015","-8,536,198"
742,20130701,3481,群創,"9,994,000","18,323,000","-8,329,000","307,000","393,000","-86,000","-1,583,000","518,000","2,101,000","-9,998,000"
743,20130701,2330,台積電,"25,849,906","39,462,311","-13,612,405","25,000","850,000","-825,000","1,517,000","3,637,000","2,120,000","-12,920,405"
